<a href="https://colab.research.google.com/github/antonionipo/Data-analysis-for-populism-in-presidential-speeches/blob/main/data_analysis_for_populism_in_presidential_speeches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data analysis for populism in presidencial speeches

Abstract
What is the most efficient way to ideologically classify a speech? Given the volume of content produced, this dissertation compares two machine learning tools for this purpose. The first is a computational learning model built using RStudio software, while the second is Chat-GPT. The classification object of this project will be official presidential speeches from Brazil (1988-2019), focusing on populism as the ideological aspect. Words characterizing relevant thematic areas will also be analyzed in a more in-depth investigation of the texts to understand the phenomenon of populism. Finally, the efficiency of these two speech classification tools into populist or non-populist will be measured by comparing their results with those obtained by a team of experts. The results obtained will contribute to the methodological debate and aim to provide a tool that facilitates this crucial work for Political Science, being also relevant to society by proposing an analysis of populism among Brazil's executive representatives, a topic of social interest.

Canto (2025)

## Research problem

What is the most efficient way to ideologically classify a speech?


## Data Understanding

Para o desenvolvimento desta pesquisa, foi utilizada a base de dados disponibilizada pelo artigo de Ricci (2021) publicado na Revista Brasileira de Ciências Sociais. Esta base contém uma classificação de discursos presidenciais oficiais brasileiros em populistas ou não populistas sob o critério de contraposição de povo e elite (Hawkins e Kaltwasser, 2017). Nesta base, foi aplicada, inicialmente, a técnica de dicionário para filtrar as potencialmente populistas, com um dicionário feito para indicar frases com palavras referentes ao povo e outro, para a elite. Apenas as que continham palavras de ambos os dicionários foram manualmente classificadas e é essa a amostra que será utilizada nesta análise, podendo ser inteiramente replicada.
É cabível ressaltar que a classificação manual feita por estes autores especialistas é tido como o padrão ouro dentro da análise de texto. Tomando estes dados como corretos, serão aplicadas diferentes técnicas de aprendizado supervisionado e não supervisionado para, observando o desempenho destas, discutir sobre suas aplicações na Ciência Política, assim como em outras áreas de conhecimento. O objetivo central é, desta maneira, a comparação, a contraposição destas diferentes formas de classificação automatizada.
Inicialmente, será feita uma análise do período pós-redemocratização brasileira (1988-2019), de maneira a obter  a pluralidade necessária de discursos, com a comparação e visualização de mudanças ocorridas tanto na frequência quanto na variedade de elementos populistas, formando uma base que contém períodos e elementos populistas e não-populistas.

Canto (2025)

**Desenho metodológico para construção da base de dados**

"Com desenho metodológico misto (mixed methods design), a alternativa que adotamos se divide em quatro etapas. Na primeira, combinamos a mineração de texto (text mining) com a abordagem de dicionário (dictionary-based approach) (Grimmer e Stewart, 2013; Lucas et al., 2015; Wilkerson e Casas, 2017; Welbers et al., 2017).10 Dividimos todos os discursos em sentenças e, a partir de um dicionário pré-definido de palavras, selecionamos as frases com potencial de expressar conteúdo populista. Nas demais etapas, avaliamos qualitativamente se os trechos selecionados de fato caracterizam conteúdo populista, ou seja, se neles o representante expressa o antagonismo entre uma elite má, conspiradora e corrupta, que usurpa o povo bom, puro e comum. Essas quatro etapas são apresentadas a seguir."

Ricci (2021)

## Data Preparation

### Instalação de pacotes

In [1]:
!pip install spacy
!python -m spacy download pt_core_news_sm
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 26.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Importação de bibliotecas

In [2]:
# Bibliotecas gerais
import pandas as pd
import re
import unicodedata
import spacy

# Bibliotecas de tratamento de dados Natural Language Processing
from nltk import word_tokenize, download
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer

# Divisão de amostra
from sklearn.model_selection import train_test_split

# Modelos Machine Learning
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Modelos Deep Learning

# Generative AI
import openai

# Métricas
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report

In [3]:
# temp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, LSTM, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import tensorflow_probability as tfp
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler

### Leitura de dados

In [4]:
# Leitura da base
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Populism/populismo_final.csv', encoding='latin-1')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,ordem,pres_nick_name,pres_label,date,frases,n.carac,nwords,povo,elite,potencial,mauricio_class,davi_class,paolo_class,discordancia,populismo
0,Sarney-I-1985-04-21-0006,210,José Sarney,Sarney-I,1985-04-21,Deus é testemunha de que eu*lhe entregaria o m...,132,23,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Sarney-I-1985-04-21-0006,211,José Sarney,Sarney-I,1985-04-21,Eterniza-se com ele a legenda do idealismo que...,167,30,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Sarney-I-1985-04-21-0006,212,José Sarney,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",196,34,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Sarney-I-1985-04-21-0006,213,José Sarney,Sarney-I,1985-04-21,"Lágrimas temos todos, das fronteiras escondida...",123,21,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Sarney-I-1985-04-21-0006,214,José Sarney,Sarney-I,1985-04-21,"Aqui estou, meus compatriotas, sob o peso de u...",84,16,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preparação do DataFrame

In [5]:
# Cópia do dataframe
populista = df.copy()

### Tratamento de dados

#### Retirar colunas irrelevantes

In [6]:
populista.drop(['id', 'ordem', 'pres_nick_name', 'n.carac', 'nwords', 'povo', 'elite', 'mauricio_class', 'davi_class', 'paolo_class'], axis=1, inplace=True)
populista.head()

,pres_label,date,frases,potencial,discordancia,populismo
0,Sarney-I,1985-04-21,Deus é testemunha de que eu*lhe entregaria o m...,0.0,0.0,0.0
1,Sarney-I,1985-04-21,Eterniza-se com ele a legenda do idealismo que...,0.0,0.0,0.0
2,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",1.0,0.0,0.0
3,Sarney-I,1985-04-21,"Lágrimas temos todos, das fronteiras escondida...",0.0,0.0,0.0
4,Sarney-I,1985-04-21,"Aqui estou, meus compatriotas, sob o peso de u...",0.0,0.0,0.0


#### Formatação de dados

In [7]:
# Convert the 'date' column to datetime objects
populista['date'] = pd.to_datetime(populista['date'], format='%Y-%m-%d')

# Convert the 'potencial', 'discordancia', and 'populismo' to Boolean
populista['potencial'] = populista['potencial'].astype(bool)
populista['discordancia'] = (populista['discordancia']).astype(bool)
populista['populismo'] = (populista['populismo']).astype(bool)

# Convert 'frases' and 'pres_label' to string
populista['frases'] = populista['frases'].astype(str)
populista['pres_label'] = populista['pres_label'].astype(str)

# Display info to verify the change
populista.info()
populista.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137447 entries, 0 to 137446
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   pres_label    137447 non-null  object        
 1   date          137447 non-null  datetime64[ns]
 2   frases        137447 non-null  object        
 3   potencial     137447 non-null  bool          
 4   discordancia  137447 non-null  bool          
 5   populismo     137447 non-null  bool          
dtypes: bool(3), datetime64[ns](1), object(2)
memory usage: 3.5+ MB


,pres_label,date,frases,potencial,discordancia,populismo
0,Sarney-I,1985-04-21,Deus é testemunha de que eu*lhe entregaria o m...,False,False,False
1,Sarney-I,1985-04-21,Eterniza-se com ele a legenda do idealismo que...,False,False,False
2,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",True,False,False
3,Sarney-I,1985-04-21,"Lágrimas temos todos, das fronteiras escondida...",False,False,False
4,Sarney-I,1985-04-21,"Aqui estou, meus compatriotas, sob o peso de u...",False,False,False


#### Tratamento de dados nulos

In [8]:
populista.isnull().sum()

,0
pres_label,0
date,0
frases,0
potencial,0
discordancia,0
populismo,0


#### Filtro de casos potenciais

In [9]:
populista = populista.loc[populista['potencial'] == True]
populista.drop(columns=['potencial'], inplace=True)
populista.head()

<ipython-input-9-53b584101beb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  populista.drop(columns=['potencial'], inplace=True)


,pres_label,date,frases,discordancia,populismo
2,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",False,False
135,Sarney-I,1985-05-02,"Aqui estão educadores, estudantes, cientistas,...",False,False
446,Sarney-I,1985-05-18,"Na manhã de 15 de janeiro, há tão poucos e den...",False,False
469,Sarney-I,1985-05-18,"Antes, estávamos juntos para, em nome do povo,...",False,False
513,Sarney-I,1985-05-24,"O Nordeste é, sobretudo, um estado de espírito.",False,False


#### Retirada da coluna discordância

A variável será usada para posterior análise comparativa

In [10]:
# Retirado e separado para posterior análise
y1 = populista['discordancia']
populista.drop(columns=['discordancia'], inplace=True)
populista.head()

,pres_label,date,frases,populismo
2,Sarney-I,1985-04-21,"Ninguém o excedeu no amor do povo, que acompan...",False
135,Sarney-I,1985-05-02,"Aqui estão educadores, estudantes, cientistas,...",False
446,Sarney-I,1985-05-18,"Na manhã de 15 de janeiro, há tão poucos e den...",False
469,Sarney-I,1985-05-18,"Antes, estávamos juntos para, em nome do povo,...",False
513,Sarney-I,1985-05-24,"O Nordeste é, sobretudo, um estado de espírito.",False


## Modelling

### Pré processamento

#### Padronziar formato das letras

In [11]:
populista['frases'] = populista['frases'].str.lower().tolist()
populista.head()

,pres_label,date,frases,populismo
2,Sarney-I,1985-04-21,"ninguém o excedeu no amor do povo, que acompan...",False
135,Sarney-I,1985-05-02,"aqui estão educadores, estudantes, cientistas,...",False
446,Sarney-I,1985-05-18,"na manhã de 15 de janeiro, há tão poucos e den...",False
469,Sarney-I,1985-05-18,"antes, estávamos juntos para, em nome do povo,...",False
513,Sarney-I,1985-05-24,"o nordeste é, sobretudo, um estado de espírito.",False


In [12]:
download('stopwords')
download('punkt_tab')
download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Lematização e Stematização

Definindo stop words, e caracteres especiais

In [13]:
nlp = spacy.load("pt_core_news_sm")

stop_words_ptbr = stopwords.words('portuguese')

def remove_accents(text):
    """
    Remove acentos, til e cedilha do texto.
    """
    text = unicodedata.normalize('NFD', text)
    text = re.sub(r'[\u0300-\u036f]', '', text)  # Remove diacríticos (acentos, til)
    text = text.replace('ç', 'c').replace('Ç', 'C')  # Substitui cedilha
    return text


Lematização e Stematização, remoção de stop words, de caracteres especiais, e vetorização-tokenização

In [14]:
class LemmaTokenizer:
    def __init__(self, stop_words=None):
        self.stop = stop_words if stop_words else []

    def __call__(self, doc):
        """
        Tokeniza e lematiza um documento de texto.
        """
        doc = remove_accents(doc.lower())  # Remove acentos e normaliza para minúsculas
        tokens = nlp(doc)  # Processa com Spacy
        return [token.lemma_ for token in tokens if token.is_alpha and token.text not in self.stop]


Aplicação da tokenização e lematização

In [15]:
lem = LemmaTokenizer(stop_words=stop_words_ptbr)
vectorizer = CountVectorizer(tokenizer=lem)

In [16]:
# Mantendo a coluna original "frases" e criando uma nova coluna para a tokenização
populista['tokenized_frases'] = populista['frases'].apply(lambda x: lem(x))

# Visualizando para verificar
print(populista[['tokenized_frases']].head(20))

                                       tokenized_frases
2     [ninguar, exceder, amor, povo, acompanhar, lon...
135   [aqui, estao, educador, estudante, cientista, ...
446   [manha, janeiro, ha, tao, pouco, denso, mês, d...
469   [antes, estava, junto, nome, povo, conquistar,...
513             [nordeste, sobretudo, estado, espirito]
663   [grande, alegria, imenso, prazer, receber, tod...
712   [visao, espiritual, porque, verifiqueir, em aq...
742   [futuro, mundo, nao, sera, marcar, rico, pobre...
783   [achar, mundo, futuro, nao, sera, balizar, ric...
834   [poder, ter, correr, regiao, rico, todo, mundo...
838   [em este, instante, dever, seguir, exemplo, ag...
853   [ontem, reuniao, so, ouvia, falar, simon, nao,...
896   [dizer, tambem, ir, repetir, aqui, rio, sao, F...
952   [canal, complementar, privilegiar, porque, col...
1419  [nao, poder, homem, publico, ter, hoje, outro,...
1423  [ir, comecar, alia, ja, comecar, realidade, de...
1495  [lembrar, promessa, rejeitar, compromisso,

### Modelos

#### Preparação para os modelos de Machine Learning

Divisão de treino e teste

In [17]:
X = populista['tokenized_frases']
y = populista['populismo']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Vetorização de dados

In [18]:
# Converter X_train e X_test para strings para o TF-IDF e RandomOverSampler
X_train_str = [' '.join(tokens) for tokens in X_train]
X_test_str = [' '.join(tokens) for tokens in X_test]

Balanceamento de dados

In [19]:
# Balanceamento de classes com RandomOverSampler para Machine Learning
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(np.array(X_train_str).reshape(-1, 1), y_train)

# Converter X_resampled de volta para uma lista de strings
X_resampled_str = [text[0] for text in X_resampled]

Vetorização estatística TF-IDF de dados

In [20]:
# Vetorização TF-IDF com dados balanceados para Machine Learning
vectorizer = TfidfVectorizer()
X_train_tfidf_resampled = vectorizer.fit_transform(X_resampled_str)
X_test_tfidf = vectorizer.transform(X_test_str)

#### Modelos de Machine Learning

Modelos Random Forest (Ensemble), Naive Bayes (Estatístico), e Suport Vector Machine (Natural Language Processing)

In [60]:
# Criação de objeto dos modelos
random_forest = RandomForestClassifier(random_state=42)  # Ensemble
svm = SVC(random_state=42) # NLP
naive_bayes = MultinomialNB() # Estatístico

# Treinar os modelos
random_forest.fit(X_train_tfidf_resampled, y_resampled);
svm.fit(X_train_tfidf_resampled, y_resampled);
naive_bayes.fit(X_train_tfidf_resampled, y_resampled);

#### Preparação para os modelos de DeepLearning

Parâmetros

In [22]:
# Parâmetros para Deep Learning
MAX_WORDS = 10000
MAX_LEN = 100
EMBEDDING_DIM = 100

Tokenizador

In [23]:
# Tokenizador para Deep Learning (reutilizando X_resampled_str)
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_resampled_str)

X_train_seq = tokenizer.texts_to_sequences(X_resampled_str)
X_test_seq = tokenizer.texts_to_sequences(X_test_str)

Padding

In [24]:
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

#### Modelos de Deep Learning

Modelos Convolutional Neural Networks (CNN), Bayesian Neural Networks (BNN), Long Short-Term Memory (RNN)

In [32]:
# Modelo Convolutional Neural Networks (CNN)
cnn_model = Sequential([
    Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_LEN),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Modelo Bayesian Neural Network (BNN) - usando Dropout como regularização bayesiana
bnn_model = Sequential([
    Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_LEN),
    LSTM(64, return_sequences=True, dropout=0.5),
    LSTM(64, dropout=0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Modelo LSTM
lstm_model = Sequential([
    Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_LEN),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Treinar modelos - Tentar com optimizer='adagrad', loss='hinge', metrics = [tf.keras.metrics.F1Score(), tf.keras.metrics.Recall()]
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.F1Score(), tf.keras.metrics.Recall()])
cnn_model.fit(X_train_pad, y_resampled, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

bnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.F1Score(), tf.keras.metrics.Recall()])
bnn_model.fit(X_train_pad, y_resampled, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.F1Score(), tf.keras.metrics.Recall()])
lstm_model.fit(X_train_pad, y_resampled, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


83/83 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - f1_score: 0.6624 - loss: 0.5520 - recall: 0.7566 - val_f1_score: 0.0879 - val_loss: 0.1895 - val_recall: 0.2500
Epoch 2/5
83/83 ━━━━━━━━━━━━━━━━━━━━ 10s 71ms/step - f1_score: 0.6665 - loss: 0.0121 - recall: 1.0000 - val_f1_score: 0.0879 - val_loss: 0.2759 - val_recall: 0.2500
Epoch 3/5
83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - f1_score: 0.6652 - loss: 0.0092 - recall: 1.0000 - val_f1_score: 0.0879 - val_loss: 0.2630 - val_recall: 0.3125
Epoch 4/5
83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - f1_score: 0.6836 - loss: 0.0020 - recall: 0.9997 - val_f1_score: 0.0879 - val_loss: 0.2525 - val_recall: 0.3125
Epoch 5/5
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - f1_score: 0.6725 - loss: 9.6090e-04 - recall: 0.9998 - val_f1_score: 0.0879 - val_loss: 0.2636 - val_recall: 0.3125
Epoch 1/5
83/83 ━━━━━━━━━━━━━━━━━━━━ 20s 167ms/step - f1_score: 0.6716 - loss: 0.6940 - recall_1: 0.6751 - val_f1_score: 0.0879 - val_loss: 0.6636 - val_recall_1: 0.0000e+00
Epoch 2/5
8

In [62]:
def evaluate_model(model, X_test, y_test, threshold=0.45):
    # Obter as previsões do modelo
    y_pred_prob = model.predict(X_test).flatten()  # Probabilidades previstas

    # Aplicar o limiar para converter probabilidades em classes binárias
    y_pred = (y_pred_prob > threshold).astype("int32")

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    return accuracy, precision, recall, f1


In [67]:
rf_results = evaluate_model(random_forest, X_test_tfidf, y_test)
svm_results = evaluate_model(svm, X_test_tfidf, y_test)
nb_results = evaluate_model(naive_bayes, X_test_tfidf, y_test)


print(f"Random Forest: F1_Score={round(rf_results[3],4)}, Recall={round(rf_results[2],4)}, Precision={round(rf_results[1],4)}, Accuracy={round(rf_results[0],4)}")
print(f"Suport Vector Machine: F1_Score={round(svm_results[3],4)}, Recall={round(svm_results[2],4)}, Precision={round(svm_results[1],4)}, Accuracy={round(svm_results[0],4)}")
print(f"Naive Bayes: F1_Score={round(nb_results[3],4)}, Recall={round(nb_results[2],4)}, Precision={round(nb_results[1],4)}, Accuracy={round(nb_results[0],4)}")

Random Forest: F1_Score=0.2222, Recall=0.125, Precision=1.0, Accuracy=0.9598
Suport Vector Machine: F1_Score=0.1176, Recall=0.0625, Precision=1.0, Accuracy=0.9569
Naive Bayes: F1_Score=0.2, Recall=0.3125, Precision=0.1471, Accuracy=0.8851


In [66]:
cnn_results = evaluate_model(cnn_model, X_test_pad, y_test)
lstm_results = evaluate_model(lstm_model, X_test_pad, y_test)
bnn_results = evaluate_model(bnn_model, X_test_pad, y_test)

print(f"Convolutional Neural Networks: F1_Score={round(cnn_results[3],4)}, Recall={round(cnn_results[2],4)}, Precision={round(cnn_results[1],4)}, Accuracy={round(cnn_results[0],4)}")
print(f"Long Short-Term Memory: F1_Score={round(lstm_results[3],4)}, Recall={round(lstm_results[2],4)}, Precision={round(lstm_results[1],4)}, Accuracy={round(lstm_results[0],4)}")
print(f"Bayesian Neural Networks: F1_Score={round(bnn_results[3],4)}, Recall={round(bnn_results[2],4)}, Precision={round(bnn_results[1],4)}, Accuracy={round(bnn_results[0],4)}")


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step
Convolutional Neural Networks: F1_Score=0.4348, Recall=0.3125, Precision=0.7143, Accuracy=0.9626
Long Short-Term Memory: F1_Score=0.4167, Recall=0.3125, Precision=0.625, Accuracy=0.9598
Bayesian Neural Networks: F1_Score=0.0879, Recall=1.0, Precision=0.046, Accuracy=0.046


#### Classificações com Generative AI

Modelos Chat GPT